In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [22]:
pd.options.display.max_rows = 1000  
pd.options.display.max_columns = 1000

In [40]:

# Read the data into a pandas dataframe 
analysis_df = pd.read_csv('Data_Engineer_Task.csv')

# Remove the first 3 rows of the dataframe as they are not needed
analysis_df = analysis_df.drop([0, 1, 2]).reset_index(drop=True)

# Drop all columns after 'COMPANY R'
analysis_df = analysis_df.loc[:, :'COMPANY R']

# Replace NaN values in the first row with negative numbers
analysis_df.iloc[0] = analysis_df.iloc[0].fillna(-1)

# Rename the columns of the dataframe
analysis_df.columns = analysis_df.iloc[0]

# Drop columns after 'COMPANY A' and before 'COMPANY B'
analysis_df = analysis_df.loc[:, :'COMPANY A'].join(analysis_df.loc[:, 'COMPANY B':])

# Drop the first row of the dataframe as it is now the column names
analysis_df = analysis_df.drop(0).reset_index(drop=True)

# Drop the first two columns of the dataframe
analysis_df = analysis_df.iloc[:, 2:]

# Drop the 5th column of the dataframe
#analysis_df = analysis_df.drop(analysis_df.columns[4], axis=1)

# Drop rows where all elements are NaN
analysis_df = analysis_df.dropna(how='all')

# Drop columns where all elements are NaN  
analysis_df = analysis_df.dropna(axis=1, how='all')

# Create a new column 'Criteria' based on the condition
analysis_df['Criteria'] = np.where(analysis_df['Momentum Criteria'] == 'Capabilities Criteria', 'Capability', 'Momentum')

# Update the 'Criteria' column for rows after 'Capabilities Criteria'
capability_start_index = analysis_df[analysis_df['Momentum Criteria'] == 'Capabilities Criteria'].index[0]
analysis_df.loc[capability_start_index:, 'Criteria'] = 'Capability'

# Remove rows where 'Momentum Criteria' is 'Capabilities Criteria'
analysis_df = analysis_df[analysis_df['Momentum Criteria'] != 'Capabilities Criteria']

# Fill down the 'Weighting' column
analysis_df['Sub_Criteria_Weighting'] = analysis_df['Weighting'].fillna(method='ffill')
analysis_df.drop('Weighting', axis=1, inplace=True)

# Create a new column 'Sub_Criteria' which is 'Momentum Criteria' but forward fill any value with a '%'
analysis_df['Sub_Criteria'] = analysis_df['Momentum Criteria'].where(~analysis_df['Momentum Criteria'].str.contains('%', na=False)).ffill()

# Remove all rows with NaN values in column '-1'
analysis_df = analysis_df.dropna(subset=[-1])

# Rename the column '-1' to 'Sub_Criteria_2'
analysis_df.rename(columns={-1: 'Sub_Criteria_2'}, inplace=True)

# Rename the column 'Weighting' to 'Sub_Criteria_2_Weighting'
analysis_df.rename(columns={'Momentum Criteria': 'Sub_Criteria_2_Weighting'}, inplace=True)


,Sub_Criteria_2_Weighting,Sub_Criteria_2,COMPANY A,COMPANY B,COMPANY C,COMPANY D,COMPANY E,COMPANY F,COMPANY G,COMPANY H,COMPANY I,COMPANY J,COMPANY K,COMPANY K,COMPANY L,COMPANY K,COMPANY M,COMPANY N,COMPANY O,COMPANY P,COMPANY Q,COMPANY R,Criteria,Sub_Criteria_Weighting,Sub_Criteria
1,25%,Market vision,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,3.0,Momentum,5%,Vision & Strategy
2,35%,Business strategy,2.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,Momentum,5%,Vision & Strategy
3,40%,Product strategy,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Momentum,5%,Vision & Strategy
5,0%,Services versus software breakdown,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,3.0,2.0,3.0,1.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,Momentum,5%,Market Focus
6,0%,ESG reporting and data management solution,3.0,2.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,1.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,Momentum,5%,Market Focus
7,80%,Category Focus (% ESG reporting and data manag...,2.0,1.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,Momentum,5%,Market Focus
8,20%,Category Focus (% ESG reporting and data manag...,2.0,2.0,0.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,0.0,2.0,1.0,3.0,2.0,2.0,1.0,2.0,Momentum,5%,Market Focus
10,40%,Consulting partners,1.0,3.0,1.0,2.0,1.0,3.0,1.0,3.0,2.0,2.0,2.0,1.0,0.0,2.0,2.0,3.0,1.0,1.0,0.0,3.0,Momentum,8%,ESG Software and Data Management Partnerships
11,0%,Consulting partner engagement,2.0,2.0,1.0,2.0,2.0,1.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,Momentum,8%,ESG Software and Data Management Partnerships
12,0%,Consulting partner relationship,0.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,Momentum,8%,ESG Software and Data Management Partnerships


In [46]:
analysis_df.columns = analysis_df.columns.astype(str)

# Combining the dataframe by merging COMPANY columns into a single column
reduced_analysis_df = analysis_df.melt(id_vars=['Sub_Criteria_2_Weighting', 'Sub_Criteria_2','Criteria','Sub_Criteria_Weighting', 'Sub_Criteria'], 
                             value_vars=[col for col in analysis_df.columns if col.startswith('COMPANY')],
                             var_name='Company', 
                             value_name='Score')




In [47]:
reduced_analysis_df = reduced_analysis_df[['Company', 'Criteria', 'Sub_Criteria', 'Sub_Criteria_Weighting', 'Sub_Criteria_2', 'Sub_Criteria_2_Weighting', 'Score']]

In [48]:
reduced_analysis_df

,Company,Criteria,Sub_Criteria,Sub_Criteria_Weighting,Sub_Criteria_2,Sub_Criteria_2_Weighting,Score
0,COMPANY A,Momentum,Vision & Strategy,5%,Market vision,25%,2.0
1,COMPANY A,Momentum,Vision & Strategy,5%,Business strategy,35%,2.0
2,COMPANY A,Momentum,Vision & Strategy,5%,Product strategy,40%,1.0
3,COMPANY A,Momentum,Market Focus,5%,Services versus software breakdown,0%,2.0
4,COMPANY A,Momentum,Market Focus,5%,ESG reporting and data management solution,0%,3.0
...,...,...,...,...,...,...,...
2395,COMPANY R,Capability,Industry Benchmarking,6%,ESG Ratings,30%,1.0
2396,COMPANY R,Capability,Industry-Specific Functionality,3%,Real estate sector,30%,1.0
2397,COMPANY R,Capability,Industry-Specific Functionality,3%,Financial sector,50%,0.0
2398,COMPANY R,Capability,Industry-Specific Functionality,3%,Other industry-specific functionality,20%,1.0


In [ ]:
pd.save_csv(reduced_analysis_df, 'reduced_analysis_df.csv')